# Introduction

## Next Steps

URL Of the OG List: https://docs.google.com/spreadsheets/d/1Mch2d3KO2Hc1n4KBpHbNVPtqnWBQ5RX6KeRCq4eOKac/edit#gid=1499993616

## The Beautiful For Loop I have too much Pride in:

```python

for row in cube_slice.iterrows():
        if rq.get(cube_slice['URL'][index]).ok == True:
            df = pd.json_normalize(rq.get(cube_slice['URL'][index]).json().get('cards').get('mainboard'), sep='_')
            df['ID'] = cube_slice['ID'][index]
            df['Cube Name'] = cube_slice['Cube Name'][index]
            df['Cubecon Type'] = 'Pre-Accepted'
            df_list.append(df)
            index+=1
        else:
            print('Error Encountered At Index:', index)
            index+=1
```

This is where the magic happens.  

We start by iterating for each *row* in our Cube Export.  We want each URL that we've created to generate a JSON response _only for that row_.  This is where we call our `[index]` value because you need to capture the specific row related to each field.  

However, before we begin, we don't want our Loop to get broken by a failed HTTP Request for information, so we call the `req.get().ok` parameter to ensure that we _start_ with a `200` response aka `True` before beginning.

We start by calling our URL related to our Index, and returning the HTTP Response of the get into JSON for parsing.  This mostly works because of the `http://cubecobra.com/cube/api/cubeJSON/` API call that returns the _entire_ webpage into JSON format.  Of the response, we're only interested in the `{cards:}` portion, and of those, the `{'mainboard':}` for analysis puroses.  However, because the response is in nested JSON, we need to flatten it because there exist really important attributes in the subsection of `{details:}` for future analysis.  Then the object is turned into a Pandas DataFrame we call "df".

However, if we just stopped there, we would lose a level of attribution when all of the DataFrames are joined together.  Currently the Cube-Level information is outside of the `{mainboard:}` attribute so we instead continue to leverage the `[index]` parameter by adding the "Cube Name" and "Cube ID" to the DataFrame, as well as noting the Type of Cube it is within the larger voting sphere of cubes.

Then the DataFrame is appended to our "df_list" and the index is increased for the next row.  If our response initially encountered a bad HTTP Request, then we print an Error Message and increase the Index so we can continue.


In [125]:
import pandas as pd
import numpy as np
import requests as rq

In [10]:
# Import The Cube List
cube_export = pd.read_csv('/Users/tpug/Jupyter Work/Magic The Gathering Projects/Cubecon_2023/Cubecon-2023/Copy of CubeCon 2023 Public Voting Dates - Pre-Accepted.csv')

In [29]:
cube_export.head()

,Cube Name and Link,Discord,Twitter Link,Twitter,Pronouns,Cube Name,URL
0,Regular Cube,Anthony#1172,ahmattox,ahmattox,"he/him, they/them",Regular Cube,https://cubecobra.com/cube/overview/regular
1,Turbo Cube,Anthony#1172,ahmattox,ahmattox,"he/him, they/them",Turbo Cube,https://cubecobra.com/cube/overview/turbo
2,Creatureless Cube,beeks#2291,rolltotweet,rolltotweet,they/them,Creatureless Cube,https://cubecobra.com/cube/overview/no-creatures
3,Dragons of Winter's Night,Chill MtG#5975,ChillMtGTV,ChillMtGTV,he/him,Dragons of Winter's Night,https://cubecobra.com/cube/overview/down
4,Eleusis,CulticCube#6756,CulticCube,CulticCube,he/him,Eleusis,https://cubecobra.com/cube/overview/culticcube


In [115]:
# Split URL Into ID, Expand into Fields
# From there, Give me only the "5th" column
# Add to DataFrame as the "ID" Field
cube_export['ID']=cube_export['URL'].str.split('/',expand=True,n=5).loc[:,5]
cube_export.head(30)

,Cube Name and Link,Discord,Twitter Link,Twitter,Pronouns,Cube Name,URL,id,ID
0,Regular Cube,Anthony#1172,ahmattox,ahmattox,"he/him, they/them",Regular Cube,https://cubecobra.com/cube/overview/regular,regular,regular
1,Turbo Cube,Anthony#1172,ahmattox,ahmattox,"he/him, they/them",Turbo Cube,https://cubecobra.com/cube/overview/turbo,turbo,turbo
2,Creatureless Cube,beeks#2291,rolltotweet,rolltotweet,they/them,Creatureless Cube,https://cubecobra.com/cube/overview/no-creatures,no-creatures,no-creatures
3,Dragons of Winter's Night,Chill MtG#5975,ChillMtGTV,ChillMtGTV,he/him,Dragons of Winter's Night,https://cubecobra.com/cube/overview/down,down,down
4,Eleusis,CulticCube#6756,CulticCube,CulticCube,he/him,Eleusis,https://cubecobra.com/cube/overview/culticcube,culticcube,culticcube
5,Petty Nobility,CulticCube#6756,CulticCube,CulticCube,he/him,Petty Nobility,https://cubecobra.com/cube/overview/pettynobility,pettynobility,pettynobility
6,Dekkaru Cube,DEKKARU#2784,GwenDekker,GwenDekker,he/him,Dekkaru Cube,https://cubecobra.com/c/dekkaru,None,None
7,Tiny Leaders,DEKKARU#2784,GwenDekker,GwenDekker,he/him,Tiny Leaders,https://cubecobra.com/cube/overview/tinyleaders,tinyleaders,tinyleaders
8,Emma Partlow's Peasant Cube,EmmaPartlow#0071,emmadpartlow,emmadpartlow,"she/her, they/them",Emma Partlow's Peasant Cube,https://cubecobra.com/cube/list/emma-peasant-c...,emma-peasant-cube-2022,emma-peasant-cube-2022
9,Old Border Foil Cube,GLHF Games#0204,glhf_games,glhf_games,he/him,Old Border Foil Cube,https://cubecobra.com/cube/list/obf,obf,obf


In [121]:
cube_slice = cube_export.loc[:, ['Cube Name', 'ID']]
# Manually Replacing wrong IDs for the Iteration
cube_slice.loc[6, ['ID']] = ['dekkaru']
cube_slice.loc[26, ['ID']] = ['UBHC']

cube_slice['URL'] = 'http://cubecobra.com/cube/api/cubeJSON/' + cube_slice['ID']
cube_slice['URL']

0        http://cubecobra.com/cube/api/cubeJSON/regular
1          http://cubecobra.com/cube/api/cubeJSON/turbo
2     http://cubecobra.com/cube/api/cubeJSON/no-crea...
3           http://cubecobra.com/cube/api/cubeJSON/down
4     http://cubecobra.com/cube/api/cubeJSON/culticcube
5     http://cubecobra.com/cube/api/cubeJSON/pettyno...
6        http://cubecobra.com/cube/api/cubeJSON/dekkaru
7     http://cubecobra.com/cube/api/cubeJSON/tinylea...
8     http://cubecobra.com/cube/api/cubeJSON/emma-pe...
9            http://cubecobra.com/cube/api/cubeJSON/obf
10        http://cubecobra.com/cube/api/cubeJSON/devoid
11    http://cubecobra.com/cube/api/cubeJSON/extrava...
12    http://cubecobra.com/cube/api/cubeJSON/creativ...
13          http://cubecobra.com/cube/api/cubeJSON/data
14    http://cubecobra.com/cube/api/cubeJSON/pulpnou...
15    http://cubecobra.com/cube/api/cubeJSON/LiveThe...
16    http://cubecobra.com/cube/api/cubeJSON/amonkar...
17    http://cubecobra.com/cube/api/cubeJSON/Mod

In [117]:
# Start with an Index of 0
index = 0

# Start with an Empty List, the most functional of Data Types
df_list = []

for row in cube_slice.iterrows():
        if rq.get(cube_slice['URL'][index]).ok == True:
            df = pd.json_normalize(rq.get(cube_slice['URL'][index]).json().get('cards').get('mainboard'), sep='_')
            df['ID'] = cube_slice['ID'][index]
            df['Cube Name'] = cube_slice['Cube Name'][index]
            df['Cubecon Type'] = 'Pre-Accepted'
            df_list.append(df)
            index+=1
        else:
            print('Error Encountered At Index:', index)
            index+=1

In [118]:
cube_mecca = pd.concat(df_list)
cube_mecca.groupby(['Cube Name']).count()

,cmc,colorCategory,finish,isUnlimited,rarity,status,tags,type_line,colors,cardID,...,details_colorcategory,details_tokens,details_image_flip,details_loyalty,ID,Cubecon Type,imgUrl,imgBackUrl,notes,markedForDelete
Cube Name,,,,,,,,,,,,,,,,,,,,,
Amonkar Desert,453,0,453,453,7,480,453,453,463,480,...,480,79,14,1,480,480,0,0,0,0
Commander extravaganza!,444,5,447,444,0,500,444,444,444,500,...,500,87,5,7,500,500,0,0,0,54
Creatureless Cube,272,0,272,272,219,450,357,272,224,450,...,450,133,23,8,450,450,0,0,219,359
Data Generated Vintage Cube,298,17,357,298,0,360,298,298,300,360,...,360,55,3,21,360,360,11,0,9,125
Dekkaru Cube,502,0,501,501,0,540,501,501,501,540,...,540,74,0,9,540,540,163,0,26,29
Derek’s Cube,331,27,472,330,0,480,330,334,333,480,...,480,78,20,20,480,480,0,0,0,29
Dragons of Winter's Night,373,0,387,373,0,432,374,373,373,432,...,432,28,25,0,432,432,2,0,0,24
Eleusis,450,3,450,450,0,450,450,450,450,450,...,450,81,19,31,450,450,0,0,0,0
Emma Partlow's Peasant Cube,445,0,445,445,0,450,445,445,445,450,...,450,67,12,5,450,450,0,0,0,0


In [120]:
# Esxport to CSV
#cube_mecca.to_csv('pre_accepted_cubes_init.csv', encoding='utf-8')

In [60]:
test_data = cube_slice.loc[0:3]

,Cube Name,ID,URL
0,Regular Cube,regular,http://cubecobra.com/cube/api/cubeJSON/regular
1,Turbo Cube,turbo,http://cubecobra.com/cube/api/cubeJSON/turbo
2,Creatureless Cube,no-creatures,http://cubecobra.com/cube/api/cubeJSON/no-crea...
3,Dragons of Winter's Night,down,http://cubecobra.com/cube/api/cubeJSON/down


In [79]:
# request_1 = rq.get(test_data['URL'][0]).json().get('cards').get('mainboard')
# Create a DataFrame that's a Normalized JSON View of a Request from the URL Parameter of test_data's [0]th Value
# Which is turned into JSON, and give me the 'Cards' Subset, and the 'Mainboard'
# Also Separate the Normalized fields with '_'
df_1 = pd.json_normalize(rq.get(test_data['URL'][0]).json().get('cards').get('mainboard'), sep='_')
df_1['ID'] = test_data['ID'][0]
df_1['Cube Name'] = test_data['Cube Name'][0]
df_1['Cubecon Type'] = 'Pre-Accepted'
df_1

,cmc,colorCategory,finish,isUnlimited,rarity,status,tags,type_line,colors,cardID,...,details_image_small,details_image_normal,details_art_crop,details_colorcategory,details_tokens,details_image_flip,details_loyalty,ID,Cube Name,Cubecon Type
0,1,None,Non-foil,False,None,Owned,[],Creature — Human Soldier,[W],251015ed-9408-4941-894a-158551ed2613,...,https://cards.scryfall.io/small/front/2/5/2510...,https://cards.scryfall.io/normal/front/2/5/251...,https://cards.scryfall.io/art_crop/front/2/5/2...,w,NaN,NaN,NaN,regular,Regular Cube,Pre-Accepted
1,3,None,Non-foil,False,None,Owned,[],Creature — Pegasus,[W],70e3a90c-1e5c-4646-b3d5-ff46d3fa7b35,...,https://cards.scryfall.io/small/front/7/0/70e3...,https://cards.scryfall.io/normal/front/7/0/70e...,https://cards.scryfall.io/art_crop/front/7/0/7...,w,NaN,NaN,NaN,regular,Regular Cube,Pre-Accepted
2,2,None,Non-foil,False,None,Owned,[],Creature — Cat,[W],27394079-924a-4fdb-8be2-f853193eca80,...,https://cards.scryfall.io/small/front/2/7/2739...,https://cards.scryfall.io/normal/front/2/7/273...,https://cards.scryfall.io/art_crop/front/2/7/2...,w,NaN,NaN,NaN,regular,Regular Cube,Pre-Accepted
3,4,None,Non-foil,False,None,Owned,[],Creature — Angel,[W],b958bcdd-d0ea-4ae0-9dd0-e6de5cf74128,...,https://cards.scryfall.io/small/front/b/9/b958...,https://cards.scryfall.io/normal/front/b/9/b95...,https://cards.scryfall.io/art_crop/front/b/9/b...,w,NaN,NaN,NaN,regular,Regular Cube,Pre-Accepted
4,4,None,Non-foil,False,None,Owned,[],Creature — Angel,[W],c47ba1fa-3ace-488b-97e6-d9f3b389c602,...,https://cards.scryfall.io/small/front/c/4/c47b...,https://cards.scryfall.io/normal/front/c/4/c47...,https://cards.scryfall.io/art_crop/front/c/4/c...,w,[e8a1b1f2-f067-4c8a-b134-4567e4d5a7c6],NaN,NaN,regular,Regular Cube,Pre-Accepted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,3,None,Non-foil,False,None,Owned,[],Enchantment,"[R, G]",7294e30f-0356-4bcc-a264-4cd46d8f13a6,...,https://cards.scryfall.io/small/front/7/2/7294...,https://cards.scryfall.io/normal/front/7/2/729...,https://cards.scryfall.io/art_crop/front/7/2/7...,m,NaN,NaN,NaN,regular,Regular Cube,Pre-Accepted
446,3,None,Non-foil,False,None,Owned,[],Instant,"[B, G]",0a16086c-5a74-45d0-8b38-e832cfbc80f7,...,https://cards.scryfall.io/small/front/0/a/0a16...,https://cards.scryfall.io/normal/front/0/a/0a1...,https://cards.scryfall.io/art_crop/front/0/a/0...,m,NaN,NaN,NaN,regular,Regular Cube,Pre-Accepted
447,3,None,Non-foil,False,None,Owned,[],Artifact Creature — Human Rogue,"[U, G]",b820f51c-e493-4bda-b1a0-11b566622249,...,https://cards.scryfall.io/small/front/b/8/b820...,https://cards.scryfall.io/normal/front/b/8/b82...,https://cards.scryfall.io/art_crop/front/b/8/b...,m,NaN,NaN,NaN,regular,Regular Cube,Pre-Accepted
448,5,None,Non-foil,False,None,Owned,[],Sorcery,"[G, U]",d25b13a4-6282-4426-8b01-9550f7d52d16,...,https://cards.scryfall.io/small/front/d/2/d25b...,https://cards.scryfall.io/normal/front/d/2/d25...,https://cards.scryfall.io/art_crop/front/d/2/d...,m,NaN,NaN,NaN,regular,Regular Cube,Pre-Accepted


In [95]:
index = 0
df_list = []

for row in test_data.iterrows():
    while index < 3:
        df = pd.json_normalize(rq.get(test_data['URL'][index]).json().get('cards').get('mainboard'), sep='_')
        df['ID'] = test_data['ID'][index]
        df['Cube Name'] = test_data['Cube Name'][index]
        df['Cubecon Type'] = 'Pre-Accepted'
        df_list.append(df)
        index +=1

In [128]:
# Initial Fields as of 6/3/23
extract = cube_mecca.loc[:,[
      'index'
    , 'Cube Name'
    , 'Cubecon Type'
    , 'cardID'
    , 'addedTmsp'
    , 'details_name'
    , 'details_full_name'
    , 'details_artist'
    , 'details_rarity'
    , 'details_color_identity'
    , 'details_colors'
    , 'details_set'
    , 'details_released_at'
    , 'details_cmc'
    , 'details_parsed_cost'
    , 'details_type'
    , 'details_elo'
    , 'details_popularity'
    , 'details_cubeCount'
    , 'details_loyalty'
    , 'details_power'
    , 'details_toughness'
]]

In [129]:
extract.head()

,index,Cube Name,Cubecon Type,cardID,addedTmsp,details_name,details_full_name,details_artist,details_rarity,details_color_identity,...,details_released_at,details_cmc,details_parsed_cost,details_type,details_elo,details_popularity,details_cubeCount,details_loyalty,details_power,details_toughness
0,0,Regular Cube,Pre-Accepted,251015ed-9408-4941-894a-158551ed2613,1572901810806,Favored Hoplite,Favored Hoplite [ths-13],Winona Nelson,uncommon,[W],...,2013-09-27,1,[w],Creature — Human Soldier,1240.8,1.633700,2268,NaN,1,2
1,1,Regular Cube,Pre-Accepted,70e3a90c-1e5c-4646-b3d5-ff46d3fa7b35,1572901810808,Trusted Pegasus,Trusted Pegasus [m20-314],Chris Rahn,common,[W],...,2019-07-12,3,"[w, 2]",Creature — Pegasus,1156.9,1.553744,2157,NaN,2,2
2,2,Regular Cube,Pre-Accepted,27394079-924a-4fdb-8be2-f853193eca80,1572901810808,Whitemane Lion,Whitemane Lion [a25-39],Zoltan Boros & Gabor Szikszai,common,[W],...,2018-03-16,2,"[w, 1]",Creature — Cat,1148.0,3.750738,5207,NaN,2,2
3,3,Regular Cube,Pre-Accepted,b958bcdd-d0ea-4ae0-9dd0-e6de5cf74128,1572901810809,Restoration Angel,Restoration Angel [mm3-20],Johannes Voss,rare,[W],...,2017-03-17,4,"[w, 3]",Creature — Angel,1366.1,18.057857,25069,NaN,3,4
4,4,Regular Cube,Pre-Accepted,c47ba1fa-3ace-488b-97e6-d9f3b389c602,1572901810809,Emeria Angel,Emeria Angel [ima-20],Jim Murray,rare,[W],...,2017-11-17,4,"[w, w, 2]",Creature — Angel,1223.7,4.370939,6068,NaN,3,3


In [130]:
extract.to_csv('pre_accepted_cubes_extract.csv', encoding='utf-8')